[View in Colaboratory](https://colab.research.google.com/github/Masum06/intention_finder/blob/master/intent_finder.ipynb)

In [1]:
!pip install git+https://github.com/mit-nlp/MITIE.git
!pip install spacy

  Cloning https://github.com/mit-nlp/MITIE.git to /tmp/pip-req-build-hMOo44
  Running setup.py bdist_wheel for mitie ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /tmp/pip-ephem-wheel-cache-EDcyij/wheels/b4/c1/21/8e7e7e14cf3211bf5c73aad0b1d76d1186fbf681f4b9ef6c06
Successfully built mitie


In [2]:
!git clone https://github.com/mit-nlp/MITIE.git

Cloning into 'MITIE'...
remote: Counting objects: 33180, done.
remote: Total 33180 (delta 0), reused 0 (delta 0), pack-reused 33180
Receiving objects: 100% (33180/33180), 10.68 MiB | 10.87 MiB/s, done.
Resolving deltas: 100% (23147/23147), done.


In [0]:
!python -m spacy download en_core_web_sm

In [3]:
!ls

datalab  MITIE


In [4]:
%%bash
cd MITIE
make MITIE-models

curl -LO https://github.com/mit-nlp/MITIE/releases/download/v0.4/MITIE-models-v0.2.tar.bz2
tar -xjf MITIE-models-v0.2.tar.bz2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   615    0   615    0     0    615      0 --:--:-- --:--:-- --:--:--  1447
100  415M  100  415M    0     0  25.9M      0  0:00:16  0:00:16 --:--:-- 33.5M


In [5]:
!ls MITIE/MITIE-models/english

binary_relations  ner_model.dat  total_word_feature_extractor.dat


In [0]:
data={
  "greet": {
    "examples" : ["hello","hey there","howdy","hello","hi","hey","hey ho"],
    "centroid" : None
  },
  "inform": {
    "examples" : [
      "i'd like something asian",
      "maybe korean",
      "what mexican options do i have",
      "what italian options do i have",
      "i want korean food",
      "i want german food",
      "i want vegetarian food",
      "i would like chinese food",
      "i would like indian food",
      "what japanese options do i have",
      "korean please",
      "what about indian",
      "i want some vegan food",
      "maybe thai",
      "i'd like something vegetarian",
      "show me french restaurants",
      "show me a cool malaysian spot"
    ],
    "centroid" : None
  },
  "deny": {
    "examples" : [
      "nah",
      "any other places ?",
      "anything else",
      "no thanks"
      "not that one",
      "i do not like that place",
      "something else please",
      "no please show other options"
    ],
    "centroid" : None
  }
}

In [7]:
!wget https://www.dropbox.com/s/0abglv8scuorkhs/glove.6B.zip

--2018-08-14 06:30:08--  https://www.dropbox.com/s/0abglv8scuorkhs/glove.6B.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0abglv8scuorkhs/glove.6B.zip [following]
--2018-08-14 06:30:08--  https://www.dropbox.com/s/raw/0abglv8scuorkhs/glove.6B.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc570708264a6dd5bb434674d89.dl.dropboxusercontent.com/cd/0/inline/AN6GrKAAHhfVmeMWit6Rc2hyPnyRci6h3vsqXWJ6fgefxPz6Zj4kDuIeR2YhPH-puqoXJLYg3iLKO-hl2cXTZcocLv9A5ukKV93AMz5rMmDD6yOhM1OMKPYMMbxLSZVSO-va6mcj5-30ZU00cZFioQNQl_oxElY_1QprGU4i6i9u0WqAz2H92kBaC__FmitqF-U/file [following]
--2018-08-14 06:30:08--  https://ucc570708264a6dd5bb434674d89.dl.dropboxusercontent.com/cd/0/inline/AN6GrKAAHhfVmeMWit6Rc2hyPnyRci6h3vsqXWJ6fgefxPz6

In [0]:
import sys, os

In [9]:
!unzip glove.6B.zip -d ./glove

Archive:  glove.6B.zip
  inflating: ./glove/glove.6B.50d.txt  
  inflating: ./glove/glove.6B.100d.txt  
  inflating: ./glove/glove.6B.200d.txt  
  inflating: ./glove/glove.6B.300d.txt  


In [10]:
!ls

datalab  glove	glove.6B.zip  MITIE


In [0]:

from mitie import *

trainer = text_categorizer_trainer("MITIE/MITIE-models/english/total_word_feature_extractor.dat")

 # same as before  - omitted for brevity
training_examples = data

for label in training_examples.keys():
  for text in training_examples[label]["examples"]:
    tokens = tokenize(text)
    trainer.add_labeled_text(tokens,label)
    
trainer.num_threads = 4
cat = trainer.train()

cat.save_to_disk("my_text_categorizer.dat")

In [0]:
input_string = "somewhere that serves chinese food"
tokens = tokenize(input_string)
predicted_label, _ = cat(tokens)

In [0]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [48]:
print "Intention: ", predicted_label
doc = nlp(u'{}'.format(input_string)) #input_string.encode('utf-8')
for chunk in doc.noun_chunks:
  if chunk.root.dep_ == u'dobj':
    print(chunk.text)

Intention:  inform
chinese food


Created with the help of: https://medium.com/rasa-blog/do-it-yourself-nlp-for-bot-developers-2e2da2817f3d